In [1]:
from main import init_system
from api.apiutils import Relation
api, reporting = init_system("/Users/ra-mit/development/discovery_proto/models/chemical/")

Loading: */Users/ra-mit/development/discovery_proto/models/chemical/*

### Help Menu

You can use the system through an **API** object. API objects are returnedby the *init_system* function, so you can get one by doing:

***your_api_object = init_system('path_to_stored_model')***

Once you have access to an API object there are a few concepts that are useful to use the API. **content** refers to actual values of a given field. For example, if you have a table with an attribute called __Name__ and values *Olu, Mike, Sam*, content refers to the actual values, e.g. Mike, Sam, Olu.

**schema** refers to the name of a given field. In the previous example, schema refers to the word__Name__ as that's how the field is called.

Finally, **entity** refers to the *semantic type* of the content. This is in experimental state. For the previous example it would return *'person'* as that's what those names refer to.

Certain functions require a *field* as input. In general a field is specified by the source name (e.g. table name) and the field name (e.g. attribute name). For example, if we are interested in finding content similar to the one of the attribute *year* in the table *Employee* we can provide the field in the following way:

field = ('Employee', 'year') # field = [<source_name>, <field_name>)
Took 0.12091517448425293 to load all data


In [2]:
reporting.num_tables

91

In [3]:
reporting.num_columns

592

In [4]:
reporting.num_content_sim_relations

340.0

In [7]:
res = api.reverse_lookup('1466093309')
for x in res: print(x)

('1466093309', 'drugbank', 'drugs_vw', 'description')


In [8]:
res = api.reverse_lookup('958135980')
for x in res: print(x)

('958135980', 'drugbank', 'drugs', 'description')


In [9]:
res = api.reverse_lookup('3131975632')
for x in res: print(x)

('3131975632', 'chembl_21', 'bio_component_sequences', 'component_type')


In [10]:
res = api.reverse_lookup('2510671983')
for x in res: print(x)

('2510671983', 'chembl_21', 'target_type', 'target_type')


In [11]:
res = api.reverse_lookup('3429407118')
for x in res: print(x)

('3429407118', 'chembl_21', 'confidence_score_lookup', 'target_mapping')


In [12]:
res = api.reverse_lookup('1278153375')
for x in res: print(x)

('1278153375', 'drugbank', 'drugs_vw', 'name')


In [13]:
res = api.reverse_lookup('1052567085')
for x in res: print(x)

('1052567085', 'drugbank', 'drugs', 'name')


In [15]:
res = api.reverse_lookup('2586135064')
for x in res: print(x)

('2586135064', 'drugbank', 'drugs_vw', 'brands')


In [16]:
res = api.reverse_lookup('2586135064')
for x in res: print(x)

('2586135064', 'drugbank', 'drugs_vw', 'brands')


In [17]:
res = api.reverse_lookup('2114051569')
for x in res: print(x)

('2114051569', 'drugbank', 'drug_brands', 'brand')


In [18]:
res = api.reverse_lookup('1352537849')
for x in res: print(x)

('1352537849', 'drugbank', 'drugs_vw', 'synonyms')


In [19]:
res = api.reverse_lookup('2488532373')
for x in res: print(x)

('2488532373', 'drugbank', 'drug_synonyms', 'synonym')


In [20]:
res = api.reverse_lookup('3429407118')
for x in res: print(x)

('3429407118', 'chembl_21', 'confidence_score_lookup', 'target_mapping')


In [21]:
res = api.reverse_lookup('2510671983')
for x in res: print(x)

('2510671983', 'chembl_21', 'target_type', 'target_type')


In [22]:
res = api.reverse_lookup('3429407118')
for x in res: print(x)

('3429407118', 'chembl_21', 'confidence_score_lookup', 'target_mapping')


In [23]:
res = api.reverse_lookup('3131975632')
for x in res: print(x)

('3131975632', 'chembl_21', 'bio_component_sequences', 'component_type')


In [24]:
res = api.reverse_lookup('2261437309')
for x in res: print(x)

('2261437309', 'chebi', 'compounds', 'chebi_accession')


In [25]:
res = api.reverse_lookup('2647114165')
for x in res: print(x)

('2647114165', 'chebi', 'vertice', 'vertice_ref')


In [27]:
res = api.reverse_lookup('2261437309')
for x in res: print(x)

('2261437309', 'chebi', 'compounds', 'chebi_accession')


In [30]:
res = api.reverse_lookup('2376510320')
for x in res: print(x)

('2376510320', 'chembl_21', 'protein_classification', 'protein_class_desc')


In [31]:
res = api.reverse_lookup('1080903695')
for x in res: print(x)

('1080903695', 'chembl_21', 'protein_family_classification', 'protein_class_desc')


In [32]:
res = api.reverse_lookup('2510671983')
for x in res: print(x)

('2510671983', 'chembl_21', 'target_type', 'target_type')


In [33]:
res = api.reverse_lookup('3131975632')
for x in res: print(x)

('3131975632', 'chembl_21', 'bio_component_sequences', 'component_type')


In [34]:
res = api.reverse_lookup('2510671983')
for x in res: print(x)

('2510671983', 'chembl_21', 'target_type', 'target_type')


In [35]:
res = api.reverse_lookup('3429407118')
for x in res: print(x)

('3429407118', 'chembl_21', 'confidence_score_lookup', 'target_mapping')


In [36]:
res = api.reverse_lookup('2488532373')
for x in res: print(x)

('2488532373', 'drugbank', 'drug_synonyms', 'synonym')


In [37]:
res = api.reverse_lookup('1352537849')
for x in res: print(x)

('1352537849', 'drugbank', 'drugs_vw', 'synonyms')


In [38]:
res = api.reverse_lookup('1080903695')
for x in res: print(x)

('1080903695', 'chembl_21', 'protein_family_classification', 'protein_class_desc')


In [39]:
res = api.reverse_lookup('2376510320')
for x in res: print(x)

('2376510320', 'chembl_21', 'protein_classification', 'protein_class_desc')


### 1.  Find columns with schema names similar to X

In [3]:
schema_name_of_interest = "ligand"
res = api.schema_name_search(schema_name_of_interest, max_results=30)
res.pretty_print_columns()

DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: Ligand InChI                  
DB: kidatabase           TABLE: KiDatabase.csv                 FIELD: Ligand ID                     
DB: kidatabase           TABLE: KiDatabase.csv                 FIELD:  Ligand Name                  
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: Ligand SMILES                 
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: Ligand InChI Key              
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: Ligand HET ID in PDB          
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: ChEMBL ID of Ligand           
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: KEGG ID of Ligand             
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: BindingDB Ligand Name         
DB: bindingdb            TABLE: BindingDB_All_clean.csv        FIELD: Link to Ligand-Target

### 2.  Find columns that contain value X

In [5]:
res = api.keyword_search("caffeine")
res.pretty_print_columns()

DB: drugbank             TABLE: drug_interactions              FIELD: description                   
DB: drugbank             TABLE: drug_interactions              FIELD: name                          
DB: chembl_21            TABLE: formulations                   FIELD: ingredient                    
DB: kidatabase           TABLE: KiDatabase.csv                 FIELD:  Ligand Name                  
DB: chembl_21            TABLE: products                       FIELD: trade_name                    


### 3.  Find columns whose content is similar to column X

In [10]:
column = ('bindingdb', 'BindingDB_All_clean.csv', 'DrugBank ID of Ligand')
drs_column = api.drs_from_raw_field(column)
similar_columns = api.similar_content_to(drs_column)
similar_columns.print_columns()

### 4.  Find tables that contain a column with the name X

In [ ]:
column_name = "ingredient"
res = api.schema_name_search(column_name)
res.print_tables()

### 5.  Find a virtual schema

In [ ]:
# Virtual schema: SMILES, CAS, name
res_smiles = api.schema_name_search("SMILES", max_results=100)
res_smiles = api.table(res_smiles)

res_CAS = api.schema_name_search("CAS", max_results = 100)
res_CAS = api.table(res_CAS)

res_name = api.schema_name_search("name", max_results = 100)
res_name = api.table(res_name)

res1 = api.intersection(res_smiles, res_CAS)
api.table(res1)
res2 = api.intersection(res1, res_name)

real_res = api.union(res1, res2)

real_res.print_tables()


### 7.  Find tables that are similar to table T

In [ ]:
table = "drug_indication"
drs_table = api.drs_from_table(table)
res = api.similar_content_to(drs_table)
res.print_tables()

### 8.  Find columns on which tables X and Y join

#### Example 1

In [ ]:
table1 = "molecule_dictionary"
drs_t1 = api.drs_from_table(table1)
table2 = "ontology"
drs_t2 = api.drs_from_table(table2)
res = api.paths_between(drs_t1, drs_t2, Relation.PKFK)
res.print_columns()

In [ ]:
res.why_id(1699088556)

In [ ]:
res.how_id(1699088556)

#### Example 2

In [ ]:
table1 = "research_companies"
drs_t1 = api.drs_from_table(table1)
table2 = "research_stem"
drs_t2 = api.drs_from_table(table2)
res = api.paths_between(drs_t1, drs_t2, Relation.PKFK)
res.print_columns()

### 9.  Find join paths between X and Y

In [ ]:
table1 = "research_stem"
drs_t1 = api.drs_from_table(table1)
table2 = "usan_stems"
drs_t2 = api.drs_from_table(table2)
res = api.paths_between(drs_t1, drs_t2, Relation.PKFK)
res.paths()

### 10. Find all tables that join on table T

#### Example 1

In [ ]:
table1 = "research_stem"
drs_t1 = api.drs_from_table(table1)
res = api.pkfk_of(drs_t1)
res.print_tables()

#### Example 2

In [ ]:
table1 = "drugs"
drs_t1 = api.drs_from_table(table1)
res = api.pkfk_of(drs_t1)
res.print_columns()

#### Example 3

In [ ]:
table1 = "drug_interactions"
drs_t1 = api.drs_from_table(table1)
res = api.pkfk_of(drs_t1)
res.print_tables()

### 11. Find all columns that can be added to table T

In [ ]:
# TODO: 
table1 = "drugs"
drs_t1 = api.drs_from_table(table1)
res = api.pkfk_of(drs_t1)
res.print_columns()

### Reporting

In [2]:
reporting.num_tables

91

In [3]:
reporting.num_columns

592

In [4]:
reporting.num_schema_sim_relations

568.0

In [5]:
reporting.num_content_sim_relations

340.0

In [6]:
reporting.num_pkfk_relations

29.0